In [4]:

import gensim
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import logging
from google.colab import drive
from googletrans import Translator

translator = Translator()

logging.basicConfig(level=logging.INFO)

drive.mount('/content/drive')

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

class WordEmbeddings:
    def __init__(self, model_path, limit=1000000):
        self.model_path = model_path
        self.limit = limit
        self.wv_model = self.load_word_embeddings()

    def load_word_embeddings(self):
        logging.info(f"Loading Word2Vec vectors from {self.model_path}")
        wv = gensim.models.KeyedVectors.load_word2vec_format(self.model_path, binary=True, limit=self.limit)
        return wv

    def save_word_embeddings_as_csv(self, output_path='/content/drive/MyDrive/vectors.csv'):
        logging.info(f"Saving Word2Vec vectors to {output_path}")
        self.wv_model.save_word2vec_format(output_path)


    def process_data(self, phrases_file, output_file='/content/drive/MyDrive/similarity_results.csv'):
        logging.info(f"Processing data from {phrases_file}")


        try:
            phrases_df = pd.read_excel(phrases_file)
        except UnicodeDecodeError:

            phrases_df = pd.read_excel(phrases_file, engine='openpyxl', encoding='latin1')


        print("Column Names:", phrases_df.columns)


        column_name = 'Phrases'
        phrase_embeddings = self.calculate_embeddings(phrases_df[column_name])

        try:
            similarity_matrix = cosine_similarity(phrase_embeddings)
        except ValueError as e:
            print(f"Error: {e}. Ensure that the Word2Vec model contains all the words in the data.")

        # similarity matrix
        np.savetxt(output_file, similarity_matrix, delimiter=",")



    def calculate_embeddings(self, phrases):
        embeddings = []
        for phrase in phrases:

            phrase = phrase.replace('?', '')
            phrase = phrase.replace("Lloyd's", '')
            phrase = phrase.replace("Kdo", '')
            phrase = phrase.replace("Whichsyndicates", '')

            words = phrase.split()


            words = [word for word in words if word.lower() not in stop_words]

            phrase_embedding = np.zeros_like(self.wv_model[words[0]])

            for word in words:
                if word in self.wv_model:
                    phrase_embedding += self.wv_model[word]
                else:
                    # Handle the case when the word is not present in the Word2Vec model
                    print(f"Warning: Word '{word}' not present in the Word2Vec model. Skipping.")
                    continue

            # Normalize the sum of word embeddings
            phrase_embedding /= np.linalg.norm(phrase_embedding)
            embeddings.append(phrase_embedding)

        return np.array(embeddings)


    def find_closest_match(self, input_phrase, phrases_file):
        input_embedding = self.calculate_embeddings([input_phrase])[0]
        phrases_df = pd.read_csv(phrases_file)
        phrase_embeddings = self.calculate_embeddings(phrases_df['Phrase'])

        # cosine similarity
        similarities = cosine_similarity([input_embedding], phrase_embeddings)[0]

        # closest match
        closest_match_index = np.argmax(similarities)
        closest_match_phrase = phrases_df.iloc[closest_match_index]['Phrase']
        distance = 1 - similarities[closest_match_index]

        return closest_match_phrase, distance

# Task 1: Init pipeline
model_path = '/content/drive/MyDrive/colabtesting/GoogleNews-vectors-negative300.bin'
word_embeddings = WordEmbeddings(model_path)
word_embeddings.save_word_embeddings_as_csv()

# Task 2: Process data
phrases_file = '/content/phrases.xlsx'
word_embeddings.process_data(phrases_file)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Column Names: Index(['Phrases'], dtype='object')
